In [41]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix

In [42]:
features_df = pd.read_parquet("data/extracted_features.parquet")

In [43]:
features_df.head()

,user_edit_frequency,editor_used,area_delta,length_delta,area,length,bounding_box_size,geometry_valid,time_since_last_edit,edit_time_of_day,...,centroid_x,centroid_y,country_count,historical_validity,tag_density,change_ratio,changeset_comment_length,source_reliability,geometry,vandalism
0,32.000000,iD 1.5.4,-597,0,0,0,1.180083e-05,0,53.874167,23,...,-63.006063,-84.906174,0,1,0.0,-1.0,0,0,"POLYGON ((-62.9969414 -84.90658619999999, -62....",True
1,32.000000,iD 1.5.4,-661,0,0,0,1.543374e-05,0,53.874167,23,...,-63.023894,-84.905413,0,1,0.0,-1.0,0,0,"POLYGON ((-62.9729527 -84.9058725, -62.9757816...",True
2,5.490196,iD 2.5.1,-6351,0,0,0,7.911717e-06,0,0.000278,1,...,-90.208055,38.524025,1,1,0.0,-1.0,25,0,"POLYGON ((-90.20804009999999 38.5251629, -90.2...",True
3,3.762032,JOSM,-60560,0,0,0,3.357874e-05,0,0.000278,20,...,-0.904047,41.657534,1,1,0.0,-1.0,22,0,"POLYGON ((-0.9073251 41.658674399999995, -0.90...",True
4,3791.000000,JOSM,-47,0,0,0,5.833380e-09,0,0.000278,20,...,-34.881462,-8.043139,1,1,0.0,-1.0,21,0,"POLYGON ((-34.8814103 -8.0431621, -34.8815028 ...",True


In [44]:
if features_df.columns.__contains__('geometry'):
    features_df.drop('geometry', axis=1, inplace=True)

In [45]:
features_df.columns = features_df.columns.str.replace(' ', '_', regex=True)
features_df.head()

,user_edit_frequency,editor_used,area_delta,length_delta,area,length,bounding_box_size,geometry_valid,time_since_last_edit,edit_time_of_day,...,bbox_y_range,centroid_x,centroid_y,country_count,historical_validity,tag_density,change_ratio,changeset_comment_length,source_reliability,vandalism
0,32.000000,iD 1.5.4,-597,0,0,0,1.180083e-05,0,53.874167,23,...,0.000566,-63.006063,-84.906174,0,1,0.0,-1.0,0,0,True
1,32.000000,iD 1.5.4,-661,0,0,0,1.543374e-05,0,53.874167,23,...,0.000786,-63.023894,-84.905413,0,1,0.0,-1.0,0,0,True
2,5.490196,iD 2.5.1,-6351,0,0,0,7.911717e-06,0,0.000278,1,...,0.004987,-90.208055,38.524025,1,1,0.0,-1.0,25,0,True
3,3.762032,JOSM,-60560,0,0,0,3.357874e-05,0,0.000278,20,...,0.003423,-0.904047,41.657534,1,1,0.0,-1.0,22,0,True
4,3791.000000,JOSM,-47,0,0,0,5.833380e-09,0,0.000278,20,...,0.000060,-34.881462,-8.043139,1,1,0.0,-1.0,21,0,True


In [46]:
X = features_df.drop('vandalism', axis=1).copy()
X.head()

,user_edit_frequency,editor_used,area_delta,length_delta,area,length,bounding_box_size,geometry_valid,time_since_last_edit,edit_time_of_day,...,bbox_x_range,bbox_y_range,centroid_x,centroid_y,country_count,historical_validity,tag_density,change_ratio,changeset_comment_length,source_reliability
0,32.000000,iD 1.5.4,-597,0,0,0,1.180083e-05,0,53.874167,23,...,0.020857,0.000566,-63.006063,-84.906174,0,1,0.0,-1.0,0,0
1,32.000000,iD 1.5.4,-661,0,0,0,1.543374e-05,0,53.874167,23,...,0.019641,0.000786,-63.023894,-84.905413,0,1,0.0,-1.0,0,0
2,5.490196,iD 2.5.1,-6351,0,0,0,7.911717e-06,0,0.000278,1,...,0.001586,0.004987,-90.208055,38.524025,1,1,0.0,-1.0,25,0
3,3.762032,JOSM,-60560,0,0,0,3.357874e-05,0,0.000278,20,...,0.009811,0.003423,-0.904047,41.657534,1,1,0.0,-1.0,22,0
4,3791.000000,JOSM,-47,0,0,0,5.833380e-09,0,0.000278,20,...,0.000097,0.000060,-34.881462,-8.043139,1,1,0.0,-1.0,21,0


In [47]:
y = features_df['vandalism'].copy()
y.head()

0    True
1    True
2    True
3    True
4    True
Name: vandalism, dtype: bool

In [49]:
X_encoded = features_df.get_dummies(X, columns=['source_reliability',
                                                'historical_validity',
                                                'country_count',
                                                'population_changed',
                                                'boundary_changed',
                                                'name_changed',
                                                'tags_modified',
                                                'tags_added',
                                                'tags_removed',
                                                'edit_time_of_day',
                                                'geometry_valid',
                                                'editor_used'])
X_encoded.head()

AttributeError: 'DataFrame' object has no attribute 'get_dummies'